# Cotiss Takehome Project

## Overview

In Australia and New Zealand alone there are over 40,000 government contracts posted each year. Most contracts have common meta data such as the date they are published, their associated category and the region in which the good/service is being procured. In addition, they contain free text that describes the details of the tender.

At Cotiss, part of our mission is to help businesses find those opportunities that are relevant for them. In order to make these contracts accessible to SMEs, it should take minimal resources to find relevant opportunities. One of the key components in this is making them searchable. This means that the content of the tender can be processed, so relevant businesses can be notified.

A key issue in searching for listings is that, often, those searching don't know what phrases to search for. An improvement for plaintext search is an autocomplete search feature that prompts the user on what they should search. You have probably experienced this when doing a Google search.

## The task

For this task, we have collated a file called _listings.json_ containing the content of thousands of public tenders posted through Government portals. Your job is to transform the data into a structure that is easier to search.

We need to improve our capabilities for our users to quickly locate tenders that matter to them. Currently our users are having a hard time to filter out irrelevant tenders and the locating tenders for their business.

User story: As a user I want to be able to effectively search for tenders that relate to my business
Acceptance Criteria
- Can efficiently search thousands of tenders
- Only shows tenders relating to the search phrase
- Recommends what phrases to search for
- Filters results by meta data

We are not asking you to implement the search component itself. The task is to create a model that will make it simpler for a search API to perform the above tasks. This project is open ended and designed to provide a platform to show your creativity, coding and problem solving skills. There is also no strict requirement on what the final solution should look like. There are no right or wrong answers so don't overthink it - just state your assumptions and justification for the model you think works best. If you have any questions, please reach out to matthew.oh@cotiss.com and ask away - we're here to help!

Your approach may also include external services in addition to python. However, if you do so, please state what service was used and how you used it. An example might be using AWS Comprehend to extract key phrases. There are also many examples for this type of application online. If you use example code, please include the required attribution.

## Output

- Once the model has been produced, it should show the output when run on the file provided. 
- Graphical analysis in order to interprets the results.
- Suggestions on how this data could be combined with the search input to produce results. For example: "We recommend using n-gram to match a searched phrase, the data returned will be a list of matched tenders and relevant meta-data".


## Submission

Before you start this project, please fork the repository and add it to your GitHub. Everything must be documented below this description, including the code. From the time you have received the project, you will have 7 days to complete it. Once you have completed it, reach out to matthew.oh@cotiss.com. We will aim to get back to you within a couple of days. Last of all, please timebox this excerice to 2-3 hours. We value your time, so don't expect a near perfect solution.

# Starting work on project

Start time: 10:06am

In [88]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from collections import Counter
from sentence_transformers import SentenceTransformer, models
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

# Short EDA to get familiar with the data


In [ ]:
#Read in the data as a dataframe
data = pd.read_json("listings.json")

#Check data characterstics
print("Number of listsings: ", len(data))
print("Column names: ", data.columns)

#Print out some random listings to get a feel of what's going on and type of text we have
for i in range(15):
    #Get random index
    index = random.randint(0, len(data))
    print()
    print(data.id[index])
    print()
    print(data.title[index])
    print()
    print(data.blurb[index])
    print()
    print(data.description[index])
    print()
    print(data.regions[index])
    print("==============")

#Seems like title and description might be easiest to work with for now. Blurb contains useful meta-information but needs a lot of cleaning, also some cases where 
#there are no blurbs (empty strings). Some cases of description and blurb being duplicates too.

# Cleaning up the titles and descriptions

In [5]:
#Cleaning the text data
def preprocess_text(text):

    #keep only alphanumeric characters and lower text
    text_no_punct = re.sub(r'[^a-zA-Z0-9]', ' ', text).lower()
    
    #Lemmatize, keep only open-class POS-words (maybe too much..) and numbers and remove stopwords
    #Lemmatization will introduce uniformity to the data
    #Closed-class words (determinters, conjunctions) tend to not hold too much useful information and might make our structures too sparse
    #TODO: Consider adding more tags
    #List of tags: https://www.learntek.org/blog/categorizing-pos-tagging-nltk-python/
    closed_classes = ['CC', 'DT', 'IN', 'PRP', 'PRP$', 'WDT', 'WP', 'WP$', "LS", "MD", "PDT", "FW", "TO"]
    stopword_list = stopwords.words("english")

    text_pos = nltk.pos_tag(text_no_punct.split())
    open_class_text = [pos_tuple[0] for pos_tuple in text_pos if pos_tuple[1] not in closed_classes and pos_tuple[0] not in stopword_list]

    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in open_class_text]
    
    #Return cleaned string
    return " ".join(lemmatized_text)

In [7]:
#Form new columns in the DF for the cleaned title and descriptions which will be used for the proposed approaches
data['cleaned_title'] = data.title.apply(lambda x: preprocess_text(x))
data['cleaned_description'] = data.description.apply(lambda x: preprocess_text(x))

#Concat the two together to increase likelihood of retrieval
#In some cases this could help if title or description contains information the other one doesn't
#Could be redundant too, as there are also duplicate cases
data['clean_title_description'] = data['cleaned_title'] + " " + data['cleaned_description']

#Double check things are looking correct
print(data['clean_title_description'].head(10))
print(len(data))

#TODO: display some of the most commonly occuring words so that I can form better search phrases for the testing. Not too familiar with the context of tendering and what services are typically offered

0    monitoring regulatory change reserve bank seek...
1    provision interpretation translation service a...
2    lao australia institute phase iii lai iii dfat...
3    raaf richmond cctv upgrade nsw scope work proj...
4    management operation catering cafeteria servic...
5    raaf townsville sewer water work qld maintenan...
6    global lightning data service procurement aim ...
7    angle park 14 dwelling construction defence ho...
8    atm 20 1280 expeditioner training service depa...
9    national priority 4 5 residual current device ...
Name: clean_title_description, dtype: object
1982


# Approach 1
## Set up a dictionary of n_grams

A dictionary where each key is an n-gram from the data and the values are indices of the listings it is found in. 

The search process would then take the input phrase from the user, clean it up as well and check which keys appear in it. Get a full list of the listing indices which hold that key, and then get the most frequently appearing listings.

Using the cleaned up text:

In [8]:
#Make a new column with chacter level n_grams for the cleaned descriptions
data['text_ngrams'] = data.clean_title_description.apply(lambda x: ["".join(gram) for gram in list(ngrams(x,n=5))])

#Function for making the dictionary structure
def form_dictionary(dataset):
    #Initialize the dictionary
    n_grams_dict = {}

    #Iterate over the given dataset
    for index in tqdm(range(len(data))):

        #Get the row of grams
        row_grams = data['text_ngrams'][index]

        #Iterate over the grams to add to dictionary
        for gram in row_grams:
            
            #Check if gram already in dict, if not initiate list for indexes and add index
            if gram not in n_grams_dict:
                n_grams_dict[gram] = []
                n_grams_dict[gram].append(index)

            else:
                n_grams_dict[gram].append(index)

    #Finally, make the values sets
    for key, value in n_grams_dict.items():
        n_grams_dict[key] = set(value)
    
    return n_grams_dict

In [9]:
#Get the dictionary structure
n_grams_dict = form_dictionary(data)

100%|██████████| 1982/1982 [00:00<00:00, 3799.67it/s]


In [37]:
#To test out how it works on some search phrases
#Give the search phrase, the dictionary structure and the top N matching listings
def search_grams_dict(search_phrase, n_grams_dict, n_results):

    #Clean the phrase too
    clean_search_phrase = preprocess_text(search_phrase)

    #Get the keys as list
    keys_list = n_grams_dict.keys()

    #Now go through the keys and if they are in the word, store indicies
    indices = []
    for key in keys_list:
        if key in clean_search_phrase:
            indices.append(list(n_grams_dict[key]))
    
    #flatten list..
    flat_indices_list = [item for sublist in indices for item in sublist]
    
    #Final step, return the n_results most frequent indicies, and thus the original entries from the data
    #TODO: return also the counts so can display for analysis
    counts = Counter(flat_indices_list)
    results = counts.most_common(n_results)
    result_indices = [result[0] for result in results]
    listing_frequencies = [result[1] for result in results]

    #Return the indices corresponding to the listings of the data
    return result_indices, listing_frequencies

In [45]:
#Funciton for displaying the results as well as some analysis
def display_results(data_indices, data, search_phrase, listing_frequencies):

    search_phrase = preprocess_text(search_phrase)
    
    #Iterate over the indices and pull from data
    listing_counter = 0

    for index in data_indices:
        print(f"Listing frequency: {listing_frequencies[listing_counter]}")
        print("Title: " + data.title[index] + "\n")
        print("Description: " + data.description[index] + "..." + "\n")

        #Check for each word in search phrase how many times it appears in the title and description of the found listting
        text_list = data['clean_title_description'][index].lower().split()
        print("Word frequencies: ")
        for word in search_phrase.split():
            print(f"{word} ({text_list.count(word)})")
        
        print("\n ======================= \n")

        listing_counter += 1

In [ ]:
#Define some search phrase
search_phrase = "building construction"

#Get indices and listing frequencies from the dicionary
data_indices, listing_frequencies = search_grams_dict(search_phrase, n_grams_dict, 5)

#Display some results
display_results(data_indices, data, search_phrase, listing_frequencies)

# Some notes on how I think the method has worked

While it does find relevant listings, it is a bit hit or miss, with the results highly dependant on the search phrase used. The search phrases need to be words which are already contained in the data, out-of-vocab words will not be discovered. The method works well with words which are "related" to each other, as in, they are more likely to be mentioned together (such as "building construction", "highway work"). If a user search is a bit more unique, let's say "environmental vacuum" the method right now tends to favour one word over the other. Listings with the word "vacuum" tend to be ignored as it is a less common word in the corpus.

The word frequencies are not that related to the listing frequency because the values (indices) in the dictionary structure were made sets in order to reduce the effect of some strange descriptions which had a lot of duplicating text.

The nice thing about this method is that it is quick, and if a user happens to misspell a word in the phrase, lets say "highway word" this should be robust as we are looking at character level n-grams, rather than full words.

## Food for thought to come back to if I have some time after implementing second approach
* Add a way to match and prioritise locations if they are put in the search phrase too, using the regions provided 
* Improve the word count ("environment" is not counted if "environmental" in text, even though it is taken into account for the actual listing rankings due to the character n-grams)

# ============================================================

# Approach 2

It would be nice if there is a way for the user to input anything they want and base the retrieval on semantic similarity. So the idea here would be to form word/sentence embeddings of the titles and descriptions of the listings using some language model (BERT for example). 

Then, a search phrase is also transformed into an embedding, and using a distance metric such as cosine similarity, pull the top X most related listings. Check both the title and description and return based on that (if both the same title and description are ranked high against the search phrase, prioritise that listing). At the same time, create a TF-IDF matrix of all descriptions of listings. Using these, for each returned listing, the user can be suggested better search phrases by listing the words of each description with the highest TF-IDF scores.

Downside here would probably be speed, as you need to compute embeddings of the search phrase. The dimensions of BERT are also large (760+) but consider reducing these using PCA for example.

Another limitation might be the token limit placed by some language models, especially in the cases of some descriptions which are really long. But an assumption here can be made that the most important and descriptive information of a description is contained in the first few sentences.

Again, work on the preprocessed text.

In [191]:
def create_embeddings(text_list, model):

    #Store the embeddings in a list
    sentence_embeddings = []
    #Iterate over the list
    for text in tqdm(text_list):
        text_embedding = model.encode(text)
        sentence_embeddings.append(text_embedding)

    return sentence_embeddings


In [ ]:
#create two models
#first one is a word embedding model
#Using a language model from the HuggingFace library. Model here can be changed depending on the task and language
#When using for the first time I believe the model needs to be downloaded which is also a time consuming step
word_embedding_model = models.Transformer('sentence-transformers/all-mpnet-base-v2', max_seq_length=256)
#The second is a pooling model which will take individual word embeddings and pool them together to form sentence embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

#Put them together
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

#Create separate lists for title and description embeddings
#This method takes some time (about 2mins for titles and 7mins for descriptions.. too long)
title_embeddings = create_embeddings(data['cleaned_title'].to_list(), model)
print(len(title_embeddings))
description_embeddings = create_embeddings(data['cleaned_description'].to_list(), model)
print(len(description_embeddings))

In [122]:
#TODO: Come back to this, didn't get dimensions matching for the search input
# #Lets reduce the embeddings using PCA (80% variance)
# pca = PCA(.80).fit(title_embeddings)
# title_embeddings_pca = pca.transform(title_embeddings)
# print("Titles reduced down to: ", len(title_embeddings_pca[0]), " dimensions")

# #Do the same for the descriptions
# # pca = PCA(.80).fit(description_embeddings)
# description_embeddings_pca = pca.transform(description_embeddings)
# print("Descriptions reduced down to: ", len(description_embeddings_pca[0]), " dimensions")

Titles reduced down to:  119  dimensions
Descriptions reduced down to:  119  dimensions


In [128]:
#Next, let's form a list of tuples, for both embeddings
#Index in list refers to index of original listing
embedding_tuples = list(zip(title_embeddings, description_embeddings))

<class 'numpy.ndarray'>


In [182]:
#Function for getting X most similar listings based on title and description embeddings
def get_listings(embedding_tuples, top_n, search_phrase_embedding, data):
    
    #make some structures to hold your results
    index_count = 0
    similarities_titles = []
    similarities_descriptions = []

    #Iterate over all embeddings
    for embedding in embedding_tuples:

        #Get cosine similarity with title
        cosine_title = cosine_similarity(search_phrase_embedding.reshape(1, -1), embedding[0].reshape(1, -1))
        similarities_titles.append([cosine_title[0][0], index_count])

        #Get cosine similarity with title
        cosine_description = cosine_similarity(search_phrase_embedding.reshape(1, -1), embedding[1].reshape(1, -1))
        similarities_descriptions.append([cosine_description[0][0], index_count])

        index_count += 1

    #now that we have both lists, order them based on their scores and take the top X
    titles_sorted = sorted(similarities_titles, key=lambda x: x[0], reverse= True)[0:top_n]
    descriptions_sorted = sorted(similarities_descriptions, key=lambda x: x[0], reverse= True)[0:top_n]

    #extract indices only
    indicies_titles = [titles[1] for titles in titles_sorted]
    indices_descriptions = [descriptions[1] for descriptions in descriptions_sorted]

    all_indices = indicies_titles + indices_descriptions

    #Count again
    counts = Counter(all_indices)
    results = counts.most_common(top_n)

    final_indices = [result[0] for result in results]
    
    #Print the resulting entries
    for index in final_indices:

        print("Title: " + data.title[index] + "\n")
        print("Description: " + data.description[index] + "..." + "\n")
        
        print("\n ======================= \n")


In [194]:
search_phrase = "I specialise in educational construction"

#Encode the search phrase too with the language model
search_phrase_embedding = model.encode(search_phrase)


In [ ]:
get_listings(embedding_tuples, 5, search_phrase_embedding, data)

# Quick notes on how I believe the second method performs

Setting up of the second approach certainly takes more time, and I am not sure how this would work in an online setting where embeddings need to be computed on the go. 

However, once the embeddings have been computed and the language model is initalized, getting the search phrase embedding is still fast and the suggested listings work surprisingly well considering no dimensionality reduction has been applied.

This approach gives the user a lot more freedom in their inputs. I played around with several wordings of work related to working on educational/school facilities and most suggestions are related to the search.

This is an advantage to the n-grams approach where wording and keywords are predefined in the dictionary and using search terms outside of that would result in poor listing selection.

Further, as multi-lingual models exist, this should allow for various languages to be used for searching, while still producing relevant listings. 
